In [31]:
from conversion import convert_train, convert_val, convert_test
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score

This part is used as a comparison to BERT pipeline

In [26]:
df_train = convert_train(pd.read_csv('data/ROC-Story-Cloze-Data.csv'))
df_val_1, df_val_2 = convert_val(pd.read_csv('data/ROC-Story-Cloze-Val.csv'))
# print("Successfully loaded csv files")

Here is the head of df_train:

InputStoryid - The original document id<br/>
Story - Context (InputSentence 1 to 4)<br/>
Quiz - InputSentence 5

In [27]:
print(df_train.head())

                               InputStoryid  \
8931   ea473442-7584-48de-9eda-69004f1b7f00   
47349  f949d814-f3ee-4c6a-906f-ba0219020632   
622    7ec40dab-5c33-45e5-a351-8365fd61b4fe   
8796   80b0e3db-6ede-4ba7-a7b9-3f847b4dc827   
8919   48b1add7-4e5c-44aa-97d5-5418573454f7   

                                                   Story  \
8931   Samantha was walking to her boyfriend's house....   
47349  Tim and his wife were getting ready for Hallow...   
622    A lady across the street was selling her dog.H...   
8796   Henry ordered some pizza.The pizza arrived lat...   
8919   Jim woke up one morning.He got ready to go to ...   

                                                    Quiz  label  
8931   She turned and ran, never telling anyone about...      1  
47349  The kids in the neighborhood all agreed their ...      1  
622                         They all died from the pain.      0  
8796   Danny tried to clean the couch with several ch...      0  
8919   Years later, George

In [28]:
# Preprocess training data
train_corpus = []
train_result = []
# count = 0

for idx, row in df_train.iterrows():
    # if count == 5: break  # For testing purpose
    
    full_text = row['Story'] + row['Quiz']
    text_list = [i for i in re.split(' |\n|\t|,|\.|!|\\?|;|:|-|–|—|~|%|_|\\|/|/|<|>|\^|\(|\)|\[|\]|\\|\'|\'|`|"', full_text) if i]
    text_processed = ' '.join(text_list).lower()
    train_corpus.append(text_processed)
    train_result.append(int(row['label']))
    
    # print(idx, text_processed)
    # count += 1
    

In [29]:
# Preprocessing CV data from df_val_1
cv1_corpus = []

for idx, row in df_val_1.iterrows():
    full_text = row['Story'] + row['Quiz']
    text_list = [i for i in re.split(' |\n|\t|,|\.|!|\\?|;|:|-|–|—|~|%|_|\\|/|/|<|>|\^|\(|\)|\[|\]|\\|\'|\'|`|"', full_text) if i]
    text_processed = ' '.join(text_list).lower()
    cv1_corpus.append(text_processed)

In [30]:
# Preprocessing CV data from df_val_1
cv2_corpus = []

for idx, row in df_val_2.iterrows():
    full_text = row['Story'] + row['Quiz']
    text_list = [i for i in re.split(' |\n|\t|,|\.|!|\\?|;|:|-|–|—|~|%|_|\\|/|/|<|>|\^|\(|\)|\[|\]|\\|\'|\'|`|"', full_text) if i]
    text_processed = ' '.join(text_list).lower()
    cv2_corpus.append(text_processed)

# Check the size of two CV corpus
assert len(cv1_corpus) == len(cv2_corpus)

In [34]:
vectorizer_tfidf = TfidfVectorizer()
train_X = vectorizer_tfidf.fit_transform(train_corpus)
train_Y = train_result
cv1_X = vectorizer_tfidf.transform(cv1_corpus)
cv2_X = vectorizer_tfidf.transform(cv2_corpus)

model = LogisticRegression(
            multi_class='multinomial',
            random_state=0,
            solver='lbfgs'
        ).fit(train_X, train_Y)

pred_cv1 = model.predict(cv1_X).tolist()
prob_cv1 = model.predict_proba(cv1_X)
pred_cv2 = model.predict(cv2_X).tolist()
prob_cv2 = model.predict_proba(cv2_X)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [38]:
# Retrieve the result
cv_pred_Y = []
for i in range(len(prob_cv1)):
    prob_1 = prob_cv1[i].tolist()
    prob_2 = prob_cv2[i].tolist()
    
    result_label = 1 if prob_1[1] > prob_2[1] else 2
    cv_pred_Y.append(result_label)
    

In [41]:
# Evaluation
cv_Y = df_val_1['AnswerRightEnding']
recall = recall_score(cv_Y, cv_pred_Y)
precision = precision_score(cv_Y, cv_pred_Y)
print()